<a href="https://colab.research.google.com/github/Alih87/Cats-and-Dogs-Classification/blob/main/YOLO_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
 import tensorflow as tf
 from keras import backend as K
 import numpy as np
 import pandas as pd
 import PIL
 from keras.layers import Input, Lambda, Conv2D
 from keras.models import load_model, Model
 import matplotlib.pyplot as plt
 import os
 from scipy import io
 from scipy import misc
 import argparse

In [5]:
def yolo_filter_boxes(box_confidence, boxes, box_class_probs, threshold = 0.6):
  box_scores = box_confidence*box_class_probs
  box_classes = K.argmax(box_scores, axis = -1)
  box_class_scores = K.max(box_scores, axis = -1)
  filtering_mask = box_class_scores >= threshold
  scores = tf.boolean_mask(box_class_scores, filtering_mask)
  boxes = tf.boolean_mask(boxes, fitering_mask)
  classes= tf.boolean_mask(box_classes, filtering_mask)

  return scores, boxes, classes

In [7]:
def iou(box1, box2):
  xi1 = np.max(box1[0], box2[0])
  xi2 = np.max(box1[2], box2[2])
  yi1 = np.max(box1[1], box2[1])
  yi2 = np.max(box1[3], box2[3])
  inter_area = max((yi2-yi1), 0)*max((xi2-xi1), 0)

  box1_area = (box1[3] - box1[0]) * (box1[2] - box1[0])
  box2_area = (box2[3] - box2[1]) * (box2[2] - box2[0])
  union_area = box1_area + box2_area - inter_area
  iou = inter_area/union_area

  return iou

In [ ]:
def yolo_non_max_suppression(scores, boxes, classes, max_boxes = 10, iou_threshold = 0.5):
  max_boxes_tensor = K.variable(max_boxes, dtype='int32')
  K.get_session().run(tf.variables_initializeer([max_boxes_tensor]))
  